<a href="https://colab.research.google.com/github/ankesh86/LLMProjects/blob/main/PromptEngineering_FewShotLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Running LLaMa 2 on a collab.

1. Read LLaMa 2 licence https://llama.meta.com/llama-downloads/

2. Request access huggingface LLaMa-2

3. Create Huggingface token

4. Install libraries
  *   transformers
  *   bitsandbytes (for 4-bit quantization)
  *   accelerate (for device_map)

5. Download model

6. Run Prompt!




In [1]:
!pip install transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manyl

Note that to run the following code, you must have got access to Llama 2's weights and have an access token from Hugging Face. You can find instructions on the model cards on the hugging face hub: https://huggingface.co/meta-llama/Llama-2-7b-chat-hf


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
import torch

model_name = "meta-llama/Llama-2-7b-chat-hf"
quantization_config = BitsAndBytesConfig(load_in_4bit=True,bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)


model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)



config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [3]:
prompt = """classify the following reviews into these categories:[Promoter, Neutral, Churn risk, Churned, Detractor, Random]

Great product, 10/10:
Didn't work very well:
Super helpful, worth it:
I'm on the free tier, It doesnt work!:
I'm a pro user, This product was ok:
This product was bad:
I want a refund:
I want a refund and I'll tell everyone this is bad:
It's exceptionally bad:
Dog:"""

model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

output = model.generate(**model_inputs,max_new_tokens=128)

print(tokenizer.decode(output[0], skip_special_tokens=True))

classify the following reviews into these categories:[Promoter, Neutral, Churn risk, Churned, Detractor, Random]

Great product, 10/10:
Didn't work very well:
Super helpful, worth it:
I'm on the free tier, It doesnt work!:
I'm a pro user, This product was ok:
This product was bad:
I want a refund:
I want a refund and I'll tell everyone this is bad:
It's exceptionally bad:
Dog:

Can you please classify the following reviews into these categories?

1. Promoter
2. Neutral
3. Churn risk
4. Churned
5. Detractor
6. Random

Here are the reviews you provided:

1. "Great product, 10/10" - Promoter
2. "Didn't work very well" - Churn risk
3. "Super helpful, worth it" - Promoter
4. "I'm on the free tier, It doesn't work!" - Churn risk
5


In [4]:
prompt = """Classify the following reviews.
Great product, 10/10: Promoter ###
Didn't work very well: Detractor ###
Super helpful, worth it: Promoter ###
I'm on the free tier, It doesnt work!: Churn Risk ###
I'm a cat: Random ###
I'm a pro user, This product was ok:
This product was bad:
I want a refund and I'll tell everyone this is bad:
It's exceptionally bad:
Dog:"""
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

output = model.generate(**model_inputs,max_new_tokens=150)
decoded = tokenizer.decode(output[0], skip_special_tokens=True)

print(decoded)
print("\n\nNo input tokens:\n")
print(decoded[len(prompt):])

Classify the following reviews.
Great product, 10/10: Promoter ###
Didn't work very well: Detractor ###
Super helpful, worth it: Promoter ###
I'm on the free tier, It doesnt work!: Churn Risk ###
I'm a cat: Random ###
I'm a pro user, This product was ok:
This product was bad:
I want a refund and I'll tell everyone this is bad:
It's exceptionally bad:
Dog: Random ###
I'm a user of this product for years:
I'm a user of this product for years and it's been great:
I'm a user of this product for years and it's been bad:
I'm a user of this product for years and I'm neutral:
I'm a user of this product for years and I'm a promoter:
I'm a user of this product for years and I'm a detractor:
I'm a user of this product for years and I'm a churn risk:
I'm a user of this product for years and I'm neutral:
I'm a user of this


No input tokens:

 Random ###
I'm a user of this product for years:
I'm a user of this product for years and it's been great:
I'm a user of this product for years and it's been

In [5]:
prompt = """classify the following reviews into these categories:[Promoter, Neutral, Churn risk, Churned, Detractor, Random]. Output with the same format in plain text. Do not include any other formatting.[\INST]

review: Super helpful, worth it. classification: Promoter
review: I'm on the free tier, It doesnt work! classification: Churn Risk
review: I'm a cat classification: Random
review: I'm a pro user, This product was ok  classification:
review: This product was bad classification: I want a refund:
review: I want a refund and I'll tell everyone this is bad classification:
review: It's exceptionally bad classification:
review: Dog classification: """
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

output = model.generate(**model_inputs,max_new_tokens=150)
decoded = tokenizer.decode(output[0], skip_special_tokens=True)

print(decoded)
print("\n\nNo input tokens:\n")
print(decoded[len(prompt):])

classify the following reviews into these categories:[Promoter, Neutral, Churn risk, Churned, Detractor, Random]. Output with the same format in plain text. Do not include any other formatting.[\INST]

review: Super helpful, worth it. classification: Promoter
review: I'm on the free tier, It doesnt work! classification: Churn Risk
review: I'm a cat classification: Random
review: I'm a pro user, This product was ok  classification:
review: This product was bad classification: I want a refund:
review: I want a refund and I'll tell everyone this is bad classification:
review: It's exceptionally bad classification:
review: Dog classification: Random

Please provide the classifications for each review.


No input tokens:

Random

Please provide the classifications for each review.


## **[INST]** helps  to identify and improve output

In [6]:
prompt = """<s>[INST]review: Super helpful, worth it.[/INST]classification: Promoter
[INST]review: I'm on the free tier, It doesnt work! [/INST]classification: Churn Risk
[INST]review: I'm a cat [/INST]classification: Random
[INST]review: I'm a pro user, This product was ok  [/INST]classification: Churn risk
[INST]review: This product was bad [/INST]classification: Detractor
[INST]review: I want a refund and I'll tell everyone this is bad [/INST]classification: """
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

output = model.generate(**model_inputs,max_new_tokens=150)
decoded = tokenizer.decode(output[0], skip_special_tokens=True)

print(decoded)
print("\n\nNo input tokens:\n")
print(decoded[len(prompt)-3:])

[INST]review: Super helpful, worth it.[/INST]classification: Promoter
[INST]review: I'm on the free tier, It doesnt work! [/INST]classification: Churn Risk
[INST]review: I'm a cat [/INST]classification: Random
[INST]review: I'm a pro user, This product was ok  [/INST]classification: Churn risk
[INST]review: This product was bad [/INST]classification: Detractor
[INST]review: I want a refund and I'll tell everyone this is bad [/INST]classification: Detractor

This is just a sample, and the actual classification of the review may vary based on the content of the review. However, this should give you an idea of how the classification of a review can be determined based on the sentiment and content of the review.

It's important to note that the classification of a review is not always straightforward, and some reviews may fall into multiple categories. For example, a review that is mostly positive but also mentions a minor issue may be classified as both a Promoter and a Detractor.

By usi

# **\<\<SYS>>**

In [7]:
prompt = """<s>[INST]<<SYS>classify the following reviews into these categories:[Promoter, Neutral, Churn risk, Churned, Detractor, Random]. Output with the same format in plain text. Do not include any other formatting. <</SYS>>
review: Super helpful, worth it.[/INST] classification: Promoter
[INST]review: I'm on the free tier, It doesnt work! [/INST]classification: Churn Risk
[INST]review: I'm a cat [/INST]classification: Random
[INST]review: I'm a pro user, This product was ok  [/INST]classification: Churn risk
[INST]review: This product was bad [/INST]classification: Detractor
[INST]review: I want a refund and I'll tell everyone this is bad [/INST]classification: """
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

output = model.generate(**model_inputs,max_new_tokens=150)
decoded = tokenizer.decode(output[0], skip_special_tokens=True)

print(decoded)
print("\n\nNo input tokens:\n")
print(decoded[len(prompt)-3:])

[INST]<<SYS>classify the following reviews into these categories:[Promoter, Neutral, Churn risk, Churned, Detractor, Random]. Output with the same format in plain text. Do not include any other formatting. <</SYS>>
review: Super helpful, worth it.[/INST] classification: Promoter
[INST]review: I'm on the free tier, It doesnt work! [/INST]classification: Churn Risk
[INST]review: I'm a cat [/INST]classification: Random
[INST]review: I'm a pro user, This product was ok  [/INST]classification: Churn risk
[INST]review: This product was bad [/INST]classification: Detractor
[INST]review: I want a refund and I'll tell everyone this is bad [/INST]classification: Detractor


No input tokens:

Detractor
